In [1]:
##run in napari env
from aicsimageio import AICSImage
from aicsimageio.writers import OmeTiffWriter
# Get an AICSImage object
img = AICSImage("file_full.ome.tif")
img.data  # returns 6D STCZYX numpy array
img.dims  # returns string "STCZYX"
img.shape  # returns tuple of dimension sizes in STCZYX order
img.shape
img.dims
first_channel_data = img.get_image_data("ZYX", T=1)

In [2]:
#get meta information from the image 
import czifile
from lxml import etree
czi = czifile.CziFile("test.image.3.czi")
czi_xml_str = czi.metadata()
print(czi_xml_str)
czi_parsed = etree.fromstring(czi_xml_str)
# this returns a list (take first element of list under presumption
# that a file can't have multiple creation dates
creation_date = czi_parsed.xpath("//CreationDate")[0]
print(creation_date.text)

<ImageDocument>
  <Metadata>
    <Experiment Version="1.2">
      <RunMode>OptimizeBeforePerformEnabled,ValidateAndAdaptBeforePerformEnabled,TimeStitchLoopedBlockEnabled</RunMode>
      <BeforeHardwareSetting>Before Exp [AF568, AF488, AF647, DAPI] Smart</BeforeHardwareSetting>
      <AfterHardwareSetting>After Exp [AF568, AF488, AF647, DAPI] Smart</AfterHardwareSetting>
      <ExperimentBlockIndex>0</ExperimentBlockIndex>
      <IsSegmented>false</IsSegmented>
      <IsStandardMode>true</IsStandardMode>
      <ImageTransferMode>MemoryMappedAndFileStream</ImageTransferMode>
      <AutoSave IsActivated="true" EnableSingleFileSave="false">
        <StorageFolder>D:\Users\Choon\230703_opto_APM\230712_opto_APM_(SFPQ)_data</StorageFolder>
        <Name>230712_63</Name>
        <IsAutoSubFolder>false</IsAutoSubFolder>
        <AlwaysAddFileIndex>true</AlwaysAddFileIndex>
        <IsAutoSubFolderExternal>true</IsAutoSubFolderExternal>
        <IsClosedOnCompletion>false</IsClosedOnCompletion>


In [3]:
#get OME Metadata from file
import bioformats
import javabridge
javabridge.start_vm(class_path=bioformats.JARS)
data = bioformats.get_omexml_metadata("test.image.3.czi")
import xml.etree.ElementTree as ET
import pandas as pd
xml_data = data
# Parse the XML
root = ET.fromstring(xml_data)
namespaces = {'ns': 'http://www.openmicroscopy.org/Schemas/OME/2016-06'} # change ns to your namespace
# Initialize a list to hold the data
cleandata = []
# Find all 'Plane' elements and iterate over them
for plane in root.findall('.//ns:Plane', namespaces):
    # Extract the attributes of the 'Plane' element and add them to the list
    cleandata.append(plane.attrib)
# Convert list of dictionaries to pandas DataFrame
df = pd.DataFrame(cleandata)
print(df)


17:33:28.831 [Thread-1] DEBUG loci.common.NIOByteBufferProvider - Using mapped byte buffer? false
17:33:28.846 [Thread-1] DEBUG loci.formats.ClassList - Could not find loci.formats.in.URLReader
java.lang.ClassNotFoundException: loci.formats.in.URLReader
	at java.base/jdk.internal.loader.BuiltinClassLoader.loadClass(BuiltinClassLoader.java:581)
	at java.base/jdk.internal.loader.ClassLoaders$AppClassLoader.loadClass(ClassLoaders.java:178)
	at java.base/java.lang.ClassLoader.loadClass(ClassLoader.java:522)
	at java.base/java.lang.Class.forName0(Native Method)
	at java.base/java.lang.Class.forName(Class.java:315)
	at loci.formats.ClassList.parseLine(ClassList.java:196)
	at loci.formats.ClassList.parseFile(ClassList.java:258)
	at loci.formats.ClassList.<init>(ClassList.java:138)
17:33:28.849 [Thread-1] DEBUG loci.formats.ClassList - Could not find loci.formats.in.SlideBook6Reader
java.lang.ClassNotFoundException: loci.formats.in.SlideBook6Reader
	at java.base/jdk.internal.loader.BuiltinClas

In [ ]:
import re

dims = img.data
match = re.search(r'T: (\d+)', dims)
if match:
    t_dim = int(match.group(1))

print(t_dim)

for x in range(t_dim):
    first_channel_data = img.get_image_data("ZYX", T=x)
    print(first_channel_data)
    # Create a new file name for each time point
    file_name = f"test.image.3_{x}.ome.tif"

    # Save the image data for the current time point
    OmeTiffWriter.save (first_channel_data,file_name)


In [ ]:
import napari
view = napari.Viewer()